In [2]:
import time
import os

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.nn.init as init
from ShanghaiTechClass import ShanghaiTech
from PIL import Image
import imageio
from toolbox import utils, metrics
#from toolbox import utils, metrics


In [3]:
vgg16 = models.vgg16(pretrained=True)

In [4]:
def conv_backend(in_channels, out_channels):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=2, dilation=2)

In [5]:
class CSRNet(nn.Module):
    
    def __init__(self):
        super(CSRNet, self).__init__()
        features = list(vgg16.features)[:23]
        self.features = nn.ModuleList(features).eval()
        self.conv1 = conv_backend(512, 512)
        init.normal_(self.conv1.weight,std=0.01)
        self.conv2 = conv_backend(512, 512)
        init.normal_(self.conv2.weight,std=0.01)
        self.conv3 = conv_backend(512, 512)
        init.normal_(self.conv3.weight,std=0.01)
        self.conv4 = conv_backend(512, 256)
        init.normal_(self.conv4.weight,std=0.01)
        self.conv5 = conv_backend(256, 128)
        init.normal_(self.conv5.weight,std=0.01)
        self.conv6 = conv_backend(128, 64)
        init.normal_(self.conv6.weight,std=0.01)
        self.convfinal = nn.Conv2d(64, 1, kernel_size=1, stride=1, padding=0, dilation=1)
        init.normal_(self.convfinal.weight,std=0.01)
        
    def forward(self,x):
        for model in self.features:
            x = model(x)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.convfinal(x))
        x = F.interpolate(x, scale_factor=8, mode='bilinear',align_corners=True)
        return x

In [6]:
model = CSRNet()
use_gpu = torch.cuda.is_available()
if use_gpu:
    model = model.cuda()

In [7]:
model

CSRNet(
  (features): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (17): Con

In [7]:
params = list(model.parameters())
print(len(params))

34


In [8]:
nb_params = 0
for layer in params:
    n = 1
    for s in list(layer.size()):
        n *= s
    nb_params += n
nb_params
# on a bien les 16.26M

16263489

In [9]:
input = 100*torch.randn(2, 3, 224, 224)
out = model(input)
print(out)

tensor([[[[0.0121, 0.0130, 0.0140,  ..., 0.0069, 0.0065, 0.0062],
          [0.0127, 0.0135, 0.0143,  ..., 0.0068, 0.0065, 0.0062],
          [0.0133, 0.0139, 0.0146,  ..., 0.0068, 0.0065, 0.0062],
          ...,
          [0.0247, 0.0241, 0.0235,  ..., 0.0041, 0.0038, 0.0034],
          [0.0247, 0.0241, 0.0235,  ..., 0.0031, 0.0027, 0.0024],
          [0.0247, 0.0241, 0.0235,  ..., 0.0020, 0.0016, 0.0013]]],


        [[[0.0129, 0.0132, 0.0136,  ..., 0.0060, 0.0059, 0.0058],
          [0.0133, 0.0136, 0.0139,  ..., 0.0061, 0.0060, 0.0058],
          [0.0138, 0.0140, 0.0143,  ..., 0.0062, 0.0061, 0.0059],
          ...,
          [0.0240, 0.0241, 0.0243,  ..., 0.0072, 0.0069, 0.0067],
          [0.0247, 0.0248, 0.0250,  ..., 0.0066, 0.0063, 0.0060],
          [0.0253, 0.0255, 0.0256,  ..., 0.0061, 0.0057, 0.0053]]]],
       grad_fn=<UpsampleBilinear2DBackward>)


In [10]:
out.size()

torch.Size([2, 1, 224, 224])

In [11]:
out.view(2,-1).sum(1).detach().numpy()

array([402.96533, 432.80798], dtype=float32)

In [12]:
train_dataset=ShanghaiTech()
#print(train_dataset.data.shape)
train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=2, shuffle=True)
#test_dataset=ShanghaiTech(train=False)
#test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=30, shuffle=True)

In [32]:
def train(model,data_loader,loss_func,optimizer,n_epochs=1):
    
    model.train(True)
    
    loss_train = np.zeros(n_epochs)
    metrics_train = np.zeros((2,n_epochs))
    
    for epoch_num in range(n_epochs):
        running_loss = 0.0
        size = 0

        for data in data_loader:
            inputs, targets = data
            counts,labels=targets[0],targets[1]
            #inputs=torch.from_numpy(inputs)
            #labels=torch.from_numpy(labels)
            #counts=torch.from_numpy(counts)
            bs = labels.size(0)
            
            if use_gpu:
                inputs.cuda()
            torch.reshape(inputs,(-1,3,224,224))
            torch.reshape(labels,(-1,1,224,224))
            inputs = inputs.type('torch.FloatTensor')
            labels = labels.type('torch.FloatTensor')
            counts = counts.type('torch.FloatTensor')
            outputs = model(inputs)


            # in your training loop:
            optimizer.zero_grad()   # zero the gradient buffers
            loss = loss_func(outputs, labels)
            loss.backward()
            optimizer.step()    # Does the update
            
            running_loss += loss
            size += bs
        epoch_loss = running_loss / size
        epoch_mae = torch.mean(torch.abs(counts-torch.sum(outputs,dim=(1,2,3)))) #running_corrects.item() / size
        epoch_mse=  torch.sqrt(torch.mean(torch.abs(counts-torch.sum(outputs,dim=(1,2,3)))**2)) #labels[:][0]
        loss_train[epoch_num] = epoch_loss
        metrics_train[0][epoch_num] = epoch_mae
        metrics_train[1][epoch_num] = epoch_mse
        
        print('Train - Loss: {:.4f} MAE: {:.4f} - MSE: {:.4f}'.format(epoch_loss, epoch_mae,epoch_mse))
        
    return loss_train, acc_train

In [ ]:
def train(args, train_loader, model, criterion, optimizer, logger, epoch,
          eval_score=None, print_freq=10, tb_writer=None):
    
    # switch to train mode
    model.train()
    meters = logger.reset_meters('train')
    meters_params = logger.reset_meters('hyperparams')
    meters_params['learning_rate'].update(optimizer.param_groups[0]['lr'])
    end = time.time()

    for i, (inputs, targets) in enumerate(train_loader):
        inputs, targets = data
        counts,labels=targets[0],targets[1]
        # print(f'{i} - {inputs.size()} - {targets.size()}')
        batch_size = inputs.size(0)

        # measure data loading time
        meters['data_time'].update(time.time() - end, n=batch_size)
       
        inputs, targets = inputs.to(args.device).requires_grad_(), targets.to(args.device)
        outputs = model(inputs)

        loss = criterion(outputs, labels)
        
        meters['loss'].update(loss.data.item(), n=batch_size)
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure accuracy and record loss
        if eval_score is not None:
            mse, pred, label = eval_score(outputs, targets)
            meters['mse'].update(mse, n=batch_size)
            meters['mae'].update(mae, n=batch_size)


        # measure elapsed time
        meters['batch_time'].update(time.time() - end, n=batch_size)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'LR {lr.val:.2e}\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=meters['batch_time'],
                   data_time=meters['data_time'], lr=meters_params['learning_rate'], loss=meters['loss'], top1=meters['mse']))


        if True == args.short_run:
            if 12 == i:
                print(' --- running in short-run mode: leaving epoch earlier ---')
                break    

   
    if args.tensorboard:
        tb_writer.add_scalar('mse/train', meters['mse'].avg, epoch)
        tb_writer.add_scalar('loss/train', meters['loss'].avg, epoch)
        tb_writer.add_scalar('learning rate', meters_params['learning_rate'].val, epoch)
       
    logger.log_meters('train', n=epoch)
    logger.log_meters('hyperparams', n=epoch)

In [33]:
model = CSRNet()
loss_func =nn.MSELoss() 
optimizer = optim.Adam(model.parameters(), lr=1e-6)
l_t, a_t = train(model, train_loader, loss_func, optimizer, n_epochs=1)

thiiiis
thiiiis
thiiiis
thiiiis
thiiiis
thiiiis
Train - Loss: 0.0000 MAE: 613.2861 - MSE: 619.1187


NameError: name 'acc_train' is not defined

In [ ]:
def test(model,data_loader):
    model.train(False)

    running_corrects = 0.0
    running_loss = 0.0
    size = 0
    
    with torch.no_grad():
        for data in data_loader:
            inputs, labels = data    
            bs = labels.size(0)

            if use_gpu:
                inputs.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            running_loss += loss
            running_corrects += ?
            size += bs

    print('Test - Loss: {:.4f} Acc: {:.4f}'.format(running_loss / size, running_corrects.item() / size))

In [ ]:
def train(args, train_loader, model, criterion, optimizer, logger, epoch,
          eval_score=None, print_freq=10, tb_writer=None):
    
    # switch to train mode
    model.train()
    meters = logger.reset_meters('train')
    meters_params = logger.reset_meters('hyperparams')
    meters_params['learning_rate'].update(optimizer.param_groups[0]['lr'])
    end = time.time()

    for i, (input, target_class) in enumerate(train_loader):
        # print(f'{i} - {input.size()} - {target_class.size()}')
        batch_size = input.size(0)

        # measure data loading time
        meters['data_time'].update(time.time() - end, n=batch_size)
       
        input, target_class = input.to(args.device).requires_grad_(), target_class.to(args.device)
        output = model(input)

        loss = criterion(output, target_class)
        
        meters['loss'].update(loss.data.item(), n=batch_size)
        
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure accuracy and record loss
        if eval_score is not None:
            acc1, pred, label = eval_score(output, target_class)
            meters['acc1'].update(acc1, n=batch_size)
            meters['confusion_matrix'].update(pred.squeeze(), label.type(torch.LongTensor))


        # measure elapsed time
        meters['batch_time'].update(time.time() - end, n=batch_size)
        end = time.time()

        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'LR {lr.val:.2e}\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Acc@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                   epoch, i, len(train_loader), batch_time=meters['batch_time'],
                   data_time=meters['data_time'], lr=meters_params['learning_rate'], loss=meters['loss'], top1=meters['acc1']))


        if True == args.short_run:
            if 12 == i:
                print(' --- running in short-run mode: leaving epoch earlier ---')
                break    

   
    if args.tensorboard:
        tb_writer.add_scalar('acc1/train', meters['acc1'].avg, epoch)
        tb_writer.add_scalar('loss/train', meters['loss'].avg, epoch)
        tb_writer.add_scalar('learning rate', meters_params['learning_rate'].val, epoch)
       
    logger.log_meters('train', n=epoch)
logger.log_meters('hyperparams', n=epoch)